In [1]:
!nvcc --version

nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2023 NVIDIA Corporation
Built on Tue_Aug_15_22:02:13_PDT_2023
Cuda compilation tools, release 12.2, V12.2.140
Build cuda_12.2.r12.2/compiler.33191640_0


In [2]:
!pip install nvcc4jupyter
%load_ext nvcc4jupyter

Detected platform "Colab". Running its setup...
Source files will be saved in "/tmp/tmpli3253jf".


Load the Dataset and unzip it

In [3]:
!rm -r *
!wget -q https://www.dropbox.com/s/i7npa4ljmt72qht/Lab3.zip
!unzip Lab3.zip
!rm Lab3.zip
!ls

Archive:  Lab3.zip
   creating: Dataset/Test/
   creating: Dataset/Test/0/
  inflating: Dataset/Test/0/input0.raw  
  inflating: Dataset/Test/0/input1.raw  
  inflating: Dataset/Test/0/MyOutput.raw  
  inflating: Dataset/Test/0/output.raw  
   creating: Dataset/Test/1/
  inflating: Dataset/Test/1/input0.raw  
  inflating: Dataset/Test/1/input1.raw  
  inflating: Dataset/Test/1/MyOutput.raw  
  inflating: Dataset/Test/1/output.raw  
   creating: Dataset/Test/10/
  inflating: Dataset/Test/10/input0.raw  
  inflating: Dataset/Test/10/input1.raw  
  inflating: Dataset/Test/10/MyOutput.raw  
  inflating: Dataset/Test/10/output.raw  
   creating: Dataset/Test/11/
  inflating: Dataset/Test/11/input0.raw  
  inflating: Dataset/Test/11/input1.raw  
  inflating: Dataset/Test/11/MyOutput.raw  
  inflating: Dataset/Test/11/output.raw  
   creating: Dataset/Test/12/
  inflating: Dataset/Test/12/input0.raw  
  inflating: Dataset/Test/12/input1.raw  
  inflating: Dataset/Test/12/MyOutput.raw  
  infl

# **GPU Code to Merge Two Vectors**

Code to me modified where there are TODO indications

In [ ]:
%%writefile lab3-new.cu
// Kernel for adding to vectors
// There are a few TODOs that you need to fill out
#include "cuda_runtime.h"
#include "device_launch_parameters.h"
#include <stdio.h>
#include <stdlib.h>
#include <string.h>
#include <time.h>
#include <math.h>

int* ImportRawInteger(char* filename, int *N)
{
	FILE* handle;
	int* val;
	int i;
	printf("Reading File %s\n", filename);
	if (filename == NULL) {
		return NULL;
	}

	handle = fopen(filename, "r");
	if (handle == NULL) {
		printf("Failed to open %s\n", filename);
		return NULL;
	}
	fscanf(handle, "%d", N);
		val = (int *) malloc(*N * sizeof(int));
	for (i = 0; i < *N; i++) {
		fscanf(handle, "%d",val+i);
	}
	fclose(handle);
	return val;

}
int ExportRawInteger(char* filename, int *val, int N)
{
	FILE* handle;
	int i;
	printf("Writing File %s\n", filename);

	handle = fopen(filename, "w");

	if (handle == NULL) {
		printf("Error writing to file: %s\n",filename);
		return 0;
	}

	fprintf(handle, "%d\n", N);
	for (i = 0; i < N; i++) {
		fprintf(handle, "%d\n", val[i]);
	}
	fclose(handle);
	return 1;
}
clock_t Time_start() {
	clock_t StartingTime;
	StartingTime=clock();
	return StartingTime;
}
int Elapsed_time(clock_t start, const char* message, int prt) {
clock_t end;
double cpu_time_used;
end = clock();
cpu_time_used = ((double) (end - start)) / CLOCKS_PER_SEC;
	if (prt == 1) { printf("%s Elapsed Time %f in mili-seconds\n", message, cpu_time_used*1000); }
	return 0;
}

__device__ int binarySearch(const int value, const int *A, const int N)
{
	// TODO: Implement a binary search that returns
	// the index where all values in A are less than
	// the given value.

	int lower = 0;
	int upper = N - 1;
	int mid;
	while (lower <= upper) {
		mid = (lower + upper) / 2;
		if (A[mid] < value) {
			lower = mid + 1;
		}
		else {
			upper = mid - 1;
		}
	}
	return lower;
}

__device__ int linearSearch(const int value, const int *A, const int N)
{
	// TODO: Implement a sequential search that returns
	// the index where all values in A are less than
	// the given value.
	int index=0;
	for (int i = 0; i < N; i++){
		if(value > a[i]){
			index++
		}
	}
	
	return index;
}

__global__ void mergeBIN(int *C, const int *A, const int *B, const int N)
{
	// TODO: Merge arrays A and B into C. To make it
	// easier you can assume the following:
	//
	// 1) A and B are both size N
	//
	// 2) C is size 2N
	//
	// 3) Both A and B are sorted arrays
	//
	// The algorithm should work as follows:
	// Given inputs A and B as follows:
	// A = [0 2 4 10]
	// B = [1 5 7 9]
	//
	// Step 1:
	// Find for each element in array A the index i that
	// would A[i] be inserted in array B or in other
	// words find the smallest j where A[i] < B[j].
	//
	// Step 2:
	// Do the same for B, but this time find the j
	// where B[i] < A[j].
	//
	// Step 3:
	// Since we know how many elements come before
	// A[i] in array A and we know how many elements
	// come before A[i] in array B, which is given by
	// are calculation of j. We should know where A[i]
	// is inserted into C, given i and j.
	//
	// This same logic can be used to find where B[i]
	// should be inserted into C. Although you will have
	// to make a minor change to handle duplicates in A
	// and B. Or in other words if A and B intersect at
	// all some values in C will be incorrect. This
	// occurs because A and B will want to put the values
	// in the same place in C.

	int index = blockIdx.x * blockDim.x + threadIdx.x;

	if (index < N) {
		int a = A[index];
		int b = binarySearch(a, B, N);
		C[index + b] = a;
	}


}
__global__ void mergeSEQ(int *C, const int *A, const int *B, const int N)
{
	// TODO: Merge arrays A and B into C. To make it
	// easier you can assume the following:
	//
	// 1) A and B are both size N
	//
	// 2) C is size 2N
	//
	// 3) Both A and B are sorted arrays
	//
	// The algorithm should work as follows:
	// Given inputs A and B as follows:
	// A = [0 2 4 10]
	// B = [1 5 7 9]
	//
	// Step 1:
	// Find for each element in array A the index i that
	// would A[i] be inserted in array B or in other
	// words find the smallest j where A[i] < B[j].
	//
	// Step 2:
	// Do the same for B, but this time find the j
	// where B[i] < A[j].
	//
	// Step 3:
	// Since we know how many elements come before
	// A[i] in array A and we know how many elements
	// come before A[i] in array B, which is given by
	// are calculation of j. We should know where A[i]
	// is inserted into C, given i and j.
	//
	// This same logic can be used to find where B[i]
	// should be inserted into C. Although you will have
	// to make a minor change to handle duplicates in A
	// and B. Or in other words if A and B intersect at
	// all some values in C will be incorrect. This
	// occurs because A and B will want to put the values
	// in the same place in C.

	for (int i = 0; i < 2*N; i++){
		int a = A[i];
		int b = linearSearch(a, B, N);
		C[i + b] = a;
	}
}




int main(int argc, char **argv) {
/*Lab3.exe  */
	int  N,i,M,mode=0;
	int* A; /*Input vector A*/
	int* B; /*Input vector B*/
	int* C; /*Output vector C*/
	int* D; /*Solution vector D*/
	int* deviceA;
	int* deviceB;
	int* deviceC;
	int prt = 0;
	int ti;
	float meanDiff = 0;
	clock_t StartingTime;
	cudaEvent_t start, stop;
	float gpu_time = 0.0f;
  char argv1[50],argv2[50],argv3[50],argv4[50],argv5[3];

	for (ti=0;ti<10;ti++){
	printf("\n\n +++++++++++++Test %i\n",ti);
	sprintf(argv1,"Dataset/Test/%i/output.raw",ti);
  sprintf(argv2,"Dataset/Test/%i/input0.raw",ti);
	sprintf(argv3,"Dataset/Test/%i/input1.raw",ti);
	sprintf(argv4,"Dataset/Test/%i/Myoutput.raw",ti);
	sprintf(argv5,"ON");

	StartingTime = Time_start();
	D = (int*)ImportRawInteger(argv1,&M);
	A = (int*)ImportRawInteger(argv2,&N);
	B = (int*)ImportRawInteger(argv3,&N);

	if (strcmp(argv5, "ON") == 0)prt = 1;
	if (strcmp(argv[1], "BIN") == 0)mode = 1;
	if (strcmp(argv[1], "SEQ") == 0)mode = 2;

	printf("Compute using %s algorithm\n",argv[1]);

	Elapsed_time(StartingTime, "Importing data and creating memory on host", prt);
	printf("The input length is %d\n", N);

	C = (int *)malloc(2 * N * sizeof(int));

	int threads = 256;
	int blocks = N / threads + ((N%threads == 0) ? 0 : 1);

	StartingTime = Time_start();
	cudaMalloc((void **)&deviceA, N * sizeof(int));
	cudaMalloc((void **)&deviceB, N * sizeof(int));
	cudaMalloc((void **)&deviceC, 2 * N * sizeof(int));
	Elapsed_time(StartingTime, "Allocating GPU memory", prt);

	StartingTime = Time_start();
	cudaMemcpy(deviceA, A, N * sizeof(int), cudaMemcpyHostToDevice);
	cudaMemcpy(deviceB, B, N * sizeof(int), cudaMemcpyHostToDevice);
	Elapsed_time(StartingTime, "Copying input memory to the GPU", prt);

	// Perform on CUDA.
	const dim3 blockSize(threads, 1, 1);
	const dim3 gridSize(blocks, 1, 1);

	cudaEventCreate(&start);
	cudaEventCreate(&stop);
	cudaEventRecord(start, 0);

	if(mode==1)	mergeBIN << < gridSize, blockSize >> >(deviceC, deviceA, deviceB, N);
	if(mode==2) mergeSEQ << < gridSize, blockSize >> >(deviceC, deviceA, deviceB, N);
	cudaDeviceSynchronize();
	cudaEventRecord(stop, 0);
	cudaEventSynchronize(stop);
	cudaEventElapsedTime(&gpu_time, start, stop);
	printf("-------->Kernel Elapsed Time*********** %f in mili-seconds\n", gpu_time);
	cudaEventDestroy(start);
	cudaEventDestroy(stop);


	StartingTime = Time_start();
	cudaMemcpy(C, deviceC, 2 * N * sizeof(int), cudaMemcpyDeviceToHost);
	Elapsed_time(StartingTime, "Copying output memory to the CPU", prt);

	StartingTime = Time_start();
	cudaFree(deviceA);
	cudaFree(deviceB);
	cudaFree(deviceC);
	Elapsed_time(StartingTime, "Freeing GPU Memory", prt);
	/*Perform Success Test*/

	for (i = 0; i < M; i++) {
		meanDiff = meanDiff + abs(C[i] - D[i]);
	}
	meanDiff = meanDiff / (float)M;
	if (meanDiff > 0.01) {
		printf("%f Failed\n",meanDiff);

	}
	else {
		printf("Passed\n");
	}
	ExportRawInteger(argv4, C, M);

	free(A);
	free(B);
	free(C);
	free(D);
	}
	return 0;
}


**Compile GPU Code**




In [5]:
!nvcc lab3-new.cu -o lab3-new.exe
!ls

Dataset  lab3-new.cu  lab3-new.exe


## Test program execution using SEQ algorithm

In [6]:
!./lab3-new.exe SEQ



 +++++++++++++Test 0
Reading File Dataset/Test/0/output.raw
Reading File Dataset/Test/0/input0.raw
Reading File Dataset/Test/0/input1.raw
Compute using SEQ algorithm
Importing data and creating memory on host Elapsed Time 0.094000 in mili-seconds
The input length is 16
Allocating GPU memory Elapsed Time 206.939000 in mili-seconds
Copying input memory to the GPU Elapsed Time 0.154000 in mili-seconds
-------->Kernel Elapsed Time*********** 152.068924 in mili-seconds
Copying output memory to the CPU Elapsed Time 0.034000 in mili-seconds
Freeing GPU Memory Elapsed Time 0.217000 in mili-seconds
Passed
Writing File Dataset/Test/0/Myoutput.raw


 +++++++++++++Test 1
Reading File Dataset/Test/1/output.raw
Reading File Dataset/Test/1/input0.raw
Reading File Dataset/Test/1/input1.raw
Compute using SEQ algorithm
Importing data and creating memory on host Elapsed Time 0.072000 in mili-seconds
The input length is 64
Allocating GPU memory Elapsed Time 0.153000 in mili-seconds
Copying input memory 

Test program execution using BIN algorithm

In [7]:
!./lab3-new.exe BIN



 +++++++++++++Test 0
Reading File Dataset/Test/0/output.raw
Reading File Dataset/Test/0/input0.raw
Reading File Dataset/Test/0/input1.raw
Compute using BIN algorithm
Importing data and creating memory on host Elapsed Time 0.078000 in mili-seconds
The input length is 16
Allocating GPU memory Elapsed Time 210.194000 in mili-seconds
Copying input memory to the GPU Elapsed Time 0.037000 in mili-seconds
-------->Kernel Elapsed Time*********** 0.287776 in mili-seconds
Copying output memory to the CPU Elapsed Time 0.020000 in mili-seconds
Freeing GPU Memory Elapsed Time 0.132000 in mili-seconds
Passed
Writing File Dataset/Test/0/Myoutput.raw


 +++++++++++++Test 1
Reading File Dataset/Test/1/output.raw
Reading File Dataset/Test/1/input0.raw
Reading File Dataset/Test/1/input1.raw
Compute using BIN algorithm
Importing data and creating memory on host Elapsed Time 0.093000 in mili-seconds
The input length is 64
Allocating GPU memory Elapsed Time 0.147000 in mili-seconds
Copying input memory to

# Installing NSYS

In [8]:
!wget https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64/nsight-systems-2023.2.3_2023.2.3.1001-1_amd64.deb
!apt update
!apt install ./nsight-systems-2023.2.3_2023.2.3.1001-1_amd64.deb
!apt --fix-broken install

--2024-09-23 16:09:52--  https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64/nsight-systems-2023.2.3_2023.2.3.1001-1_amd64.deb
Resolving developer.download.nvidia.com (developer.download.nvidia.com)... 152.195.19.142
Connecting to developer.download.nvidia.com (developer.download.nvidia.com)|152.195.19.142|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 317705436 (303M) [application/x-deb]
Saving to: ‘nsight-systems-2023.2.3_2023.2.3.1001-1_amd64.deb’

nsight-systems-2023 100%[===================>] 302.99M   204MB/s    in 1.5s    

2024-09-23 16:09:54 (204 MB/s) - ‘nsight-systems-2023.2.3_2023.2.3.1001-1_amd64.deb’ saved [317705436/317705436]

Get:1 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,626 B]
Hit:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease
Get:3 http://security.ubuntu.com/ubuntu jammy-security InRelease [129 kB]
Hit:4 http://archive.ubuntu.com/ubuntu jammy InR

Trace BIN algorithm using NSIGHT

In [9]:
!nsys profile --trace=cuda -d 20 --stats=true --force-overwrite true -o lab3-test-bin_seq  ./lab3-new.exe BIN
!ls



 +++++++++++++Test 0
Reading File Dataset/Test/0/output.raw
Reading File Dataset/Test/0/input0.raw
Reading File Dataset/Test/0/input1.raw
Compute using BIN algorithm
Importing data and creating memory on host Elapsed Time 0.064000 in mili-seconds
The input length is 16
Allocating GPU memory Elapsed Time 553.842000 in mili-seconds
Copying input memory to the GPU Elapsed Time 0.042000 in mili-seconds
-------->Kernel Elapsed Time*********** 0.249952 in mili-seconds
Copying output memory to the CPU Elapsed Time 0.025000 in mili-seconds
Freeing GPU Memory Elapsed Time 0.125000 in mili-seconds
Passed
Writing File Dataset/Test/0/Myoutput.raw


 +++++++++++++Test 1
Reading File Dataset/Test/1/output.raw
Reading File Dataset/Test/1/input0.raw
Reading File Dataset/Test/1/input1.raw
Compute using BIN algorithm
Importing data and creating memory on host Elapsed Time 0.092000 in mili-seconds
The input length is 64
Allocating GPU memory Elapsed Time 0.143000 in mili-seconds
Copying input memory to

# Upload NSYS report

In [13]:
from google.colab import files
files.download('/content/lab3-test-bin_seq.nsys-rep')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

## Save NSIGHT trace into Lab3Results-BIN.txt

## Trace SEQ algorithm using NSYS

In [11]:
!nsys profile --trace=cuda -d 20 --stats=true --force-overwrite true -o lab3-test-seq_seq  ./lab3-new.exe SEQ
!ls



 +++++++++++++Test 0
Reading File Dataset/Test/0/output.raw
Reading File Dataset/Test/0/input0.raw
Reading File Dataset/Test/0/input1.raw
Compute using SEQ algorithm
Importing data and creating memory on host Elapsed Time 0.062000 in mili-seconds
The input length is 16
Allocating GPU memory Elapsed Time 575.157000 in mili-seconds
Copying input memory to the GPU Elapsed Time 0.045000 in mili-seconds
-------->Kernel Elapsed Time*********** 0.248960 in mili-seconds
Copying output memory to the CPU Elapsed Time 0.030000 in mili-seconds
Freeing GPU Memory Elapsed Time 0.148000 in mili-seconds
Passed
Writing File Dataset/Test/0/Myoutput.raw


 +++++++++++++Test 1
Reading File Dataset/Test/1/output.raw
Reading File Dataset/Test/1/input0.raw
Reading File Dataset/Test/1/input1.raw
Compute using SEQ algorithm
Importing data and creating memory on host Elapsed Time 0.095000 in mili-seconds
The input length is 64
Allocating GPU memory Elapsed Time 0.195000 in mili-seconds
Copying input memory to

# Upload NSYS report

In [12]:
from google.colab import files
files.download('/content/lab3-test-seq_seq.nsys-rep')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

## Save NSIGHT trace into Lab3Results-SEQ.txt



## Also save your version of the notebook Lab3.ipynb